# Document Segmentation using OpenCV: Isolating the Race/Ethnicity Column in 1950 Census Population Schedules 

This notebook employs computer vision techniques from the OpenCV python module to isolate table elements within 1950s Census Population Schedule page images. With certain predetermined information about the tables structure within each document, namely the relative positions of lines, it is able to crop certain table cells for further analysis. The primary computer vision technique is called a morphological transform, in which a black and white image is redrawn according to rules based on the value of a each pixel and its neighbors. Each pixel in the new image is either black or white, depending upon the black and white value of the pixels that surrounded it in the original image. We use what is called a structuring element, also known as a kernel, to decide which neighboring pixels are important for the new pixel.
There are many other helper steps surrounding these various morphological transforms, including the follow techniques:

* Removing the border or padding around the page
* Thresholding the image such that its color pixels are replaced with only black or white pixels
* Skeletonizing the image, to be discussed later
* Detecting lines with a Hough Lines Transform
* Rotating the image to make the page precisely vertical
* Detecting horizontal and vertical lines using array accumulations
* Optimizing the alignment of image with a form template through magnication and translation transforms

We will discuss each step in the sections below, explaining both the code and its function within our overall workflow.

## Perspective on Development Process

You should know, for your own perspective, that the process of creating this image processing workflow took place over several weeks and was highly iterative and messy. Reading through the code here it would be easy to think that the author created the solution step-by-step, from beginning to end, with a plan in mind. In fact, I tried many different strategies to segment this census form, eventually settling on one that is a mixture of two different approaches. Even then the notebook was a mess, with a lot of discarded code and tweaks that were no longer necessary. I then made major revisions in order to clean up the process for publication and better explanation of the techniques. Many steps were complete failures in the first attempt, requiring careful debugging and consultation with documentation. Be kind to yourself when performing this kind of interative, messy work. Not many people can simply sit down and compose such a workflow that performs accurately the first time, myself included. Refining the approach is part of the process. 

Most of the steps here are based on a [older census-related project](http://blog.ncsa.uiuc.edu/drupal/sites/default/files/eScience.pdf) from a team at NCSA at UIUC and I received some guidance from them in implementing those steps. Their approach worked in many ways, but my forms had a little more variation in the vertical layout that required an additional feature extraction for vertical alignment of a template. I'll explain later.

## Project Goal

The goal is for this image processing is to extract an array of cells image objects for the demographic cells within the population schedule page. In a separate notebook we will apply handwritten character recognition to those cell images, in order to recognize the letter "W" representing a white person. Our study of Japanese-American displacement during and after World War II will be supported by this work, in order to focus the manual work of transcribing census information about Japanese-American households.

## Setup

### Dependencies

This notebook uses OpenCV, SciKit-Image, and Numpy python modules. OpenCV is also an operating system package that can be installed separately. While there are PyPi modules that purport to install OpenCV, the best route I have found is to install Open CV on an operating system and use the Open CV python modules cited in the PIP lines below. I have not had any luck installing OpenCV within a Jupyter Hub environment within which I do not have superuser permissions.

Note: Numpy is likely already installed in your Jupyter environment, but if not you must install it too.

### Constants

This first block of code sets up some constant values we will use later. Some color values are established that we will use as shorthand later when drawing on images. Then we have our "template" for the form lines. These are two arrays, one containing the offset of the vertical lines and another with the pixel offsets of the horizontal lines. In both cases these offsets correspond to the center of the line. Of course one array describe offsets on the horizontal axis and the other along the vertical. Note that these offsets are at half the scale of the original NARA Census images. When working with the images the first step will be to shrink them to half-scale.

Note that the template was created using an image editor. We first reduced the scale by half in each dimension. Then we measured the X pixel offset of each vertical line and the Y pixel offset of each vertical line, in order. The resulting numbers are entered in the two lists below. You can also make sure a template at a midway point in this workflow. I'll include a note there to point out that approach.

In [ ]:
GREEN = (34,255,13)
RED = (255,36,12)
BLUE = (0,0,255)

# Form lines template
# NOTE: horizontal lines only cover main table area. Some offsets estimated from surrounding offsets
vert_lines = [93,124,145,173,223,274,327,369,397,646,737,762,806,842,885,939,1047,1081,1114,1153,1208,1256,1271,1472,1508,1764,1804,1850,1856,1892,1910,1931,1964]
horiz_lines = [299,320,594,611,642,673,704,735,766,797,830,861,893,924,955,988,1018,1050,1082,1113,1144,1175,1208,1241,1270,1303,1333,1364,1395,1427,1459,1490,1522,1554]

### Helper and Debug Functions

Since we are building a complex image processing workflow, we are going to want to check our progress visually at various points. Often the best way to do this is to draw some finding directly on the image in question and view partial results. The first two functions allow us to conveniently view images as Jupyter notebook output. Another is there to show a histogram of grayscale values to understand the distribution of gray values within the image.

In [ ]:
import cv2  # this is OpenCV
import matplotlib.pyplot as plt
import skimage
import numpy as np
import math

# Notebook-friendly OpenCV image display
def viewGray(image):
    #reduced = cv2.resize(image, None, fx=0.3, fy=0.3, interpolation=cv2.INTER_LINEAR)
    plt.imshow(image, cmap='gray')
    plt.show()

def view(image, txt=''):
    #reduced = cv2.resize(image, None, fx=0.3, fy=0.3, interpolation=cv2.INTER_LINEAR)
    plt.figure(figsize = (10,10))
    plt.imshow(image, cmap='gray')
    plt.figtext(0.5, 0.01, txt, wrap=True, horizontalalignment='center', fontsize=12)
    plt.show()

def hist(image):
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    plt.figure()
    plt.title("Grayscale Histogram")
    plt.xlabel("Bins")
    plt.ylabel("# of Pixels")
    plt.plot(hist)
    plt.xlim([0, 256])
    plt.show()


### Workflow Functions

The first step in building our image workflow is to create some Python functions that present the problem and the result that we want. Since most image processing is composed of a series of linear steps, I'd like to create a procedural function that captures each step along the way. That is the currently almost empty `extract()` function below. Another function will load our sample images, call this function, and display the results.

In [ ]:
import glob
import re
import cv2

def run(path, page_range=(2, 16), debug=False):
    for f in sorted(glob.glob(f'{path}/*')):
        pagestr = re.search(r'-(\d+).jpeg', f).group(1)
        if int(pagestr) not in range(page_range[0], page_range[1]):
            continue
        image = cv2.imread(f)
        cells = extract(image, f, debug=debug)

def extract(image, filename, debug=False):
    result = []
    if debug:
        view(image, filename)
    return result

You can see that our `run()` function takes a path parameter for the folder of page images. It can also take optional page range and debug parameters. The page range will allow us to test our approach on a subset of the images, as needed. Often a technique will fail in certain cases. When that happens we need to refine it with that page image in mind. It saves a lot of time to target our tests in those situations. The debug flag or parameter controls whether the extract function will produce and display test images for various steps. The debug views create extra work for the computer and clutter our notebook output, which isn't necessary once we get our workflow completed. You can see that in the rudimentary extract function we check the debug flag before presenting a view of the current image.

Let's run our workflow on the first page image.

In [ ]:
run('pages', page_range=(2,3), debug=True)

You should see a rendering of the original page two image above. The matplotlib module has added some pixel notation around the horizontal and vertical axis. In addition the view function adds a label. In this case it is simply the filename.

Have a good look at this image and the other same page images that are included in the `pages` directory. You will notice that the main table of people is in the center of the page, with a tall set of column headers or labels above it. This table takes up most of the horizontal space, but it is surrounded vertically by a large page header and a lower section with different information. In addition there is a subtle difference you may note between various pages.. The main table is sometimes above the `Notes` field and sometimes below it. In fact, while the main table has a consistent shape by itself, it is in different vertical positions on different pages. Our workflow will have to take this into account. However, we begin with some preparation steps that are frequently used in the processing of digitized images.

## Grayscale and Reduce Size

Our first step, like many others, will remove extraneous information from the image. Our image is loaded as a color image, with three integers between 0 and 255 to represent the blue, green, and red component values for each pixel. Since our document is black and white, the separate color information is useless. We convert the image to grayscale, meaning that each pixel is represented by one integer between 0 and 255 that indicates its brightness. We also don't need the massive resolution of the original image and will work more efficiently with an image that is reduced to one quarter of the original pixels, or half the size in each dimension. OpenCV makes it straightforward to perform both of these operations. Run the code below and view the modified image.

In [ ]:
def extract(image, filename, debug=False):
    result = []
    height, width, depth = image.shape
    height = int(height/2)
    width = int(width/2)
    image = cv2.resize(image, (width, height))

    grayimage = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    if debug:
        view(grayimage, filename)
    return result

In [ ]:
run('pages', page_range=(2,3), debug=True)

Note that the image looks exactly the same, only the pixel notations on the axis, provided by matplotlib, have changed. However, our representation of the image as a data structure in memory has been reduced by 2/3 in the color dimension, and another 3/4 in the space dimensions. We are now working with image data is that 1/12 of the original's data size.



## Removing the Mat

The mat in this case refers to the mat on which the document was placed for scanning. You can see it in the image above as a black area surrounding the white paper page. That black area isn't adding any information that helps us process the page contents and will distract algorithms that focus on the black ink areas. Fortunately we have a technique to remove it. We start by detecting it as the largest contiguous black shape, creating a "mask" or a stencil that captures that shape. Stencil data has the same dimensions as the image, a two dimensional array of numbers. However, each value is only a 0 or 1, making it a sort of binary image.

We will define a separate function to create the stencil in a number of steps.

The first step is to "threshold" the image. This reduces the pixel values of the whole image to either 0 or 1, based on whether the prior pixel value is above or below a threshold value. In our case we are using a special OpenCV function, `cv2.threshold()`, to calculate an appropriate threshold value based on the pixel values of the entire image. Since I did not want the black mat to be included in this threshold calculation, I sliced out the center of the image and fed that into the threshold calculation. This means that the quantity of black in the mat region will not influence the threshold we use.

In [ ]:
def get_border_stencil(grayimage, debug=False):
    # You can use a Python slice operation to crop and image object.
    height, width = grayimage.shape
    middlegrayimg = grayimage[int(height/4):int(height*3/4), int(width/4):int(width*3/4)]
    if debug:
        view(middlegrayimg, "middle gray image sliced from full size")

    # Note that the threshold function returns a threshold number and a binary image based upon that number.
    # In this line we only keep the threshold number that was calculated using the "Otsu Algorithm".
    threshold = cv2.threshold(middlegrayimg, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[0]
    if debug:
        print(f'OpenCV calculated a threshold of {threshold} in the get_border_stencil() function')

    # Now we will apply our Otsu threshold number to a blurred version of the whole grayscale image.
    blurgray = cv2.blur(grayimage, (6,6))
    if debug:
        view(blurgray, "blurred")

    # Note that we save the second return value this time, the binary image obtained using the threshold from the prior step.
    binary_img = cv2.threshold(blurgray, threshold, 255, cv2.THRESH_BINARY)[1]
    if debug:
        view(binary_img, "thresholded")

def extract(image, filename, debug=False):
    result = []
    height, width, depth = image.shape
    height = int(height/2)
    width = int(width/2)
    image = cv2.resize(image, (width, height))
    grayimage = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

    # Adding in the function call to get the stencil
    get_border_stencil(grayimage, debug=debug)
    return result

run('pages', page_range=(2,3), debug=True)

Now that we have a binary image, where every pixel is either black or white, we can use an Open CV operation to find contours of white regions in the image. This function returns a list of contours, starting with the outermost and largest shape that was found. In our case we are looking for the contour or shape of the page itself. This ***should*** be the largest shape in the image. Our debug code creates a color copy of the image with the detected shape outlined in green.

Note: We will remove debug lines as they become less relevant to the current work.

Now that we have the contour, we can construct a stencil as an array the corresponds to the image dimensions. We create a new array filled with zeroes first, then fill the contour's region with ones. Open CV has a helpful function for drawing or filling contours.

In [ ]:
def get_border_stencil(grayimage, debug=False):
    # You can use a Python slice operation to crop and image object.
    height, width = grayimage.shape
    middlegrayimg = grayimage[int(height/4):int(height*3/4), int(width/4):int(width*3/4)]

    # Note that the threshold function returns a threshold number and a binary image based upon that number.
    # In this line we only keep the threshold number that was calculated using the "Otsu Algorithm".
    threshold = cv2.threshold(middlegrayimg, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[0]
    if debug:
        print(f'OpenCV calculated a threshold of {threshold} in the get_border_stencil() function')

    blurgray = cv2.blur(grayimage, (6,6))
    binary_img = cv2.threshold(blurgray, threshold, 255, cv2.THRESH_BINARY)[1]

    # NOTE: Here is the new code to detect the contours of the white regions.
    cnts = cv2.findContours(binary_img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if debug:
        debug_image = cv2.cvtColor(grayimage,cv2.COLOR_GRAY2BGR)
        cv2.drawContours(debug_image, cnts[0], -1, GREEN, 4)
        view(debug_image, "largest contour detected by morphological transform")

    stencil = np.zeros(binary_img.shape).astype(binary_img.dtype)
    cv2.drawContours(stencil, cnts[0], -1, 1, thickness=-1)
    if debug:
        view(stencil, "stencil")
    return stencil

run('pages', page_range=(2,3), debug=True)

At this point it is always wise to check on how well your steps work on the rest of the sample images..

In [ ]:
run('pages', page_range=(2,17), debug=True)

You should see a clean stencil shape for each page. If a clean outer page shape is not detected properly, you may have to perform additional steps to make page images more uniform, for instance eliminating printed or handwritten black areas that overlap with the mat. The blur steps takes care of most of that by mixing some of the surrounding white into any stray black lines.

Next we use our stencil in the `extract()` function to remove the mat. First we will threshold our image again using the same method as before, although this time we will also ask that the resulting binary image be inverted. This makes the printed ink white and the page paper black. Then we perform a bitwise AND operation to only allow white (ink) within the area of the stencil that is also white (ones). A bitwise AND operation will consider each pixel location, making a new image with a 1 wherever there is a 1 in both the image and the mask (stencil). Since the stencil has black in the mat area, those pixels will always be black. Inside the white of the stencil, white pixels will be preserved as is.

In [1]:
def get_border_stencil(grayimage, debug=False):
    # You can use a Python slice operation to crop and image object.
    height, width = grayimage.shape
    middlegrayimg = grayimage[int(height/4):int(height*3/4), int(width/4):int(width*3/4)]

    # Note that the threshold function returns a threshold number and a binary image based upon that number.
    # In this line we only keep the threshold number that was calculated using the "Otsu Algorithm".
    threshold = cv2.threshold(middlegrayimg, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[0]
    if debug:
        print(f'OpenCV calculated a threshold of {threshold} in the get_border_stencil() function')

    # Now we will apply our Otsu threshold number to a blurred version of the whole grayscale image.
    blurgray = cv2.blur(grayimage, (6,6))
    if debug:
        view(blurgray, "blurred")

    # Note that we save the second return value this time, the binary image obtained using the threshold from the prior step.
    binary_img = cv2.threshold(blurgray, threshold, 255, cv2.THRESH_BINARY_INV)[1]
    if debug:
        view(binary_img, "thresholded")
    
    kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT,(1,5))
    binary_img = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernel1)
    kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT,(5,1))
    binary_img = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernel2)
    binary_inverted_img = cv2.bitwise_not(binary_img)
    if debug:
        view(binary_inverted_img, "inverted")
    cnts = cv2.findContours(binary_inverted_img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if debug:
        debug_image = cv2.cvtColor(grayimage,cv2.COLOR_GRAY2BGR)
        cv2.drawContours(debug_image, cnts[0], -1, GREEN, 4)
        view(debug_image, "contours")

    stencil = np.zeros(binary_img.shape).astype(binary_img.dtype)
    cv2.drawContours(stencil, cnts[0], -1, 1, thickness=-1)
    if debug:
        view(stencil, "stencil")
    return stencil

In [ ]:
def extract(image, filename, debug=False):
    result = []
    height, width, depth = image.shape
    height = int(height/2)
    width = int(width/2)
    image = cv2.resize(image, (width, height))
    grayimage = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    stencil = get_border_stencil(grayimage, debug=False)

    # Otsu threshold and invert image in one step. Once again threshold based on middle of image.
    middlegrayimg = grayimage[int(height/4):int(height*3/4), int(width/4):int(width*3/4)]
    threshold = cv2.threshold(middlegrayimg, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[0]
    thresh = cv2.threshold(grayimage, threshold, 255, cv2.THRESH_BINARY_INV)[1]
    if debug:
        view(thresh, "inverted binary image")

    # Apply the border mask with a bitwise AND operation
    thresh = cv2.bitwise_and(thresh, thresh, mask=stencil)
    if debug:
        view(thresh, "inverted binary image with mat removed")
    
    return result

run('pages', page_range=(2,3), debug=True)

## Skeletonize or the Thinning Operation

Skeletonize is a highly descriptive technical term for an image processing operation. It takes a binary image like ours and removes ink from the narrower sides of shapes, thinning them until each shape is only one pixel wide. It kind of looks like just the bones within the shapes. If you did skeletonize the shape of a person, it would look kind of like bones except that the skull would be reduced to a vertical line, not a solid.

Single pixel width shapes are helpful when we want to distinguish hardwritten scribbles from the long straight lines that make up the paper census form. It is another way of removing extraneous information (shape or line thickness) from our image in order to perform the next step, which will be detecting long lines and then rotating our image so that those lines are mostly perfectly horizontal or vertical.

Lucky for us, Skeletonize is a single line of Open CV code. You can compare a region before and after the operation in the debug views.

In [ ]:
def extract(image, filename, debug=False):
    result = []
    height, width, depth = image.shape
    height = int(height/2)
    width = int(width/2)
    image = cv2.resize(image, (width, height))
    grayimage = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    stencil = get_border_stencil(grayimage, debug=False)

    # Otsu threshold and invert image in one step. Once again threshold based on middle of image.
    middlegrayimg = grayimage[int(height/4):int(height*3/4), int(width/4):int(width*3/4)]
    threshold = cv2.threshold(middlegrayimg, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[0]
    thresh = cv2.threshold(grayimage, threshold, 255, cv2.THRESH_BINARY_INV)[1]

    # Apply the border mask with a bitwise AND operation
    thresh = cv2.bitwise_and(thresh, thresh, mask=stencil)
    if debug:
        view(thresh[int(height/2)-500:int(height/2)+500, int(width/2)-500:int(width/2)+500], "inverted binary image (zoomed in)")
    
    # The zero specifies Zhuang Suen algorithm to the thinning operation, as per NCSA paper
    skeleton = cv2.ximgproc.thinning(thresh, 0)
    if debug:
        view(skeleton[int(height/2)-500:int(height/2)+500, int(width/2)-500:int(width/2)+500], "the skeleton (zoomed in)")

    return result

run('pages', page_range=(2,3), debug=True)

## Unrotate the Image

The straight lines in our skeleton image are printed lines from the census form. During scanning the paper pages can be skewed slightly clockwise or counter-clockwise. By applying some line-based operations, we can unskew or unrotate the page image and more or less align the form with the axes of the image. This will help later as we try to fit our form template to the lines on a particular image, since all of the significant line "ink" will either be in a single column or a single row within our image pixel array..

Let's start by added a function call to our `extract()` function that will determine the angle by which we need to rotate our image. This angle is expressed in radians. (360 degrees is 2π radians.)

In [ ]:
def extract(image, filename, debug=False):
    result = []
    height, width, depth = image.shape
    height = int(height/2)
    width = int(width/2)
    image = cv2.resize(image, (width, height))
    grayimage = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    stencil = get_border_stencil(grayimage, debug=False)

    # Otsu threshold and invert image in one step. Once again threshold based on middle of image.
    middlegrayimg = grayimage[int(height/4):int(height*3/4), int(width/4):int(width*3/4)]
    threshold = cv2.threshold(middlegrayimg, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[0]
    binary_inverted_img = cv2.threshold(grayimage, threshold, 255, cv2.THRESH_BINARY_INV)[1]

    # Apply the border mask with a bitwise AND operation
    binary_inverted_img = cv2.bitwise_and(binary_inverted_img, binary_inverted_img, mask=stencil)
    if debug and False:
        view(binary_inverted_img, "inverted binary image without mat")
    
    # The zero specifies Zhuang Suen algorithm to the thinning operation, as per NCSA paper
    skeleton = cv2.ximgproc.thinning(binary_inverted_img, 0)
    if debug and False:
        view(skeleton[int(height/2)-500:int(height/2)+500, int(width/2)-500:int(width/2)+500], "the skeleton (zoomed in)")

    radians = get_radians_to_unrotate(skeleton, debug=debug)
    if debug:
        print(f"The rotate angle for {filename} is {radians*180/np.pi} degrees.")

    # We want to fix the rotation in all of our versions of the image..
    skeleton = rotate_image(skeleton, radians)
    binary_inverted_img = rotate_image(binary_inverted_img, radians)
    image = rotate_image(image, radians)


    return result

Now we have to implement a function to detect the proper angle to unrotate our census form. In order to do this we first find the angles of all of the longer straight lines in the image. Computer vision has an algorithm for this, called the Hough Lines Transform. Then we select only those lines that are relatively close to horizontal, i.e. their angles are close to π/2 radians.
We then make subgroups of these lines based on their exact angles, putting them into bins that evenly divide that range of angles we have collected. We then select the bin with the most lines in it. In this way we remove lines that are outliers and focus on the main group of similar horizontal lines. The average angle of the lines in this bin is the angle is what consider to be horizontal within the page. Now we simply have to rotate the page such that it's horizontal angle is the same as the our image's X axis. The difference between these angles is our image horizontal angle minus the page angle, or π/2 - 𝛕, where 𝛕 is the angle we found in the page (The tau variable in our code). This rotation correction is usually small, but will have a sizable impact on later steps.

#### More about the Hough Lines Transform

For more information about the Hough Lines Transform, see the explanation from the [Open CV tutorial](https://docs.opencv.org/3.4/d3/de6/tutorial_js_houghlines.html) on the subject. You can also find many good explanations for this algorithm on the web. Once reason that we reduced the size of the image to 1/4 of the original pixels is that it makes this algorithm much more efficient.

### Rotate Image Function

This function makes some calculations requires to rotate the image around its center by the given number of radians. It calculates the center of the image, then computes a "tranform", i.e. a matrix math operation, that will rotate the image around the center by a given number of degrees. There is some math there to convert the radians used earlier into degrees.

In [ ]:
def get_radians_to_unrotate(skeleton, debug=False):
    debug_image = None
    if debug:
        debug_image = cv2.cvtColor(skeleton,cv2.COLOR_GRAY2BGR)

    min = np.pi/2 - np.pi/40  # only angles close to horizontal
    max = np.pi/2 + np.pi/40
    lines = cv2.HoughLines(skeleton, 1, np.pi / 180, 300, 0, 0, 0, 0)  
    thetas = []
    if lines is not None:
        for i in range(0, len(lines)):
            rho = lines[i][0][0]
            theta = lines[i][0][1]
            if min < theta < max:
                thetas.append(theta)

            # Our rotation function only needs theta values (line angles in radians)
            # However, we use the calculations below to draw lines for debug
            if debug:
                a = math.cos(theta)
                b = math.sin(theta)
                x0 = a * rho
                y0 = b * rho
                pt1 = (int(x0 + 2500*(-b)), int(y0 + 2500*(a)))
                pt2 = (int(x0 - 2500*(-b)), int(y0 - 2500*(a)))
                color = BLUE
                if min < theta < max:
                    color = GREEN
                cv2.line(debug_image, pt1, pt2, color, 3, cv2.LINE_AA)

    # Now we group the angles in a histogram and pick the largest group.
    # The hist variable captures the number of lines in each bin produced by the histogram function.
    # The edges variable gives us the angles that define the boundary values of the bins.
    hist, edges = np.histogram(thetas, bins='auto')
    bin_idx = 0
    for i in range(0, len(hist)):
        if hist[i] > hist[bin_idx]:
            bin_idx = i
    thetas = [ x for x in thetas if edges[bin_idx] < x <= edges[bin_idx+1]]
    
    # All of this debug code is just to draw the main bin angles in red in the image center.
    # Often the angles are identical and overlap, becoming one red line.
    if debug:
        for t in thetas:
            a = math.cos(t)
            b = math.sin(t)
            halfway_down = debug_image.shape[1] / 2
            x0 = a * halfway_down
            y0 = b * halfway_down
            pt1 = (int(x0 + 2500*(-b)), int(y0 + 2500*(a)))
            pt2 = (int(x0 - 2500*(-b)), int(y0 - 2500*(a)))
            cv2.line(debug_image, pt1, pt2, RED, 3, cv2.LINE_AA)
        view(debug_image, "Houghlines in blue and green, with green being close to horizontal")

    tau = np.average(thetas)
    result = np.pi/2 - tau
    return result

def rotate_image(image, radians):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, radians*180/np.pi, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result

Okay, that was a lot code and explanation. Let's look at some output. In this case it is instructive to look at the output from each page image, as that shows the variability in Hough Lines Transform output and why we need to be so selective of which lines we take into consideration. The debug images show the lines detect by the HoughLines Transform in blue and green. The green lines are the ones that we selected as being close to horizontal. Additional red lines in the middle of the image show all of the angles that were in the largest bin in our histogram, i.e. the largest grouping of green line angles. Often the lines in that bin share approximately the same angle, giving the appearance of a single red line. Have a good look at the debug images and reflect on why some images show so many blue lines. Despite the variations between images, did the code produce red lines that approximate the horizontal lines with respect to the printed page?

From your understanding of the Hough Lines Transform, how many more lines would have been detected if we had not removed the mat at the edge of the image?

In [ ]:

run('pages', page_range=(2,17), debug=True)

## Reflecting on our Image Data Array

Let's take a moment to consider the current state of the array of binary numbers that represent our image. Each pixel is either on or off, either a one or a zero. All of the shapes and lines in our image have been thinned until they are one pixel wide, the skeleton. Then we rotated the skeleton so that for the most part the printed horizontal lines of the form align with the horizontal axis of our image. This also means that most of the printed vertical lines align with our vertical axis.

If we now were to add up the ones in every row in the image array, we would see much larger numbers in the rows where we have horizontal lines. Other shapes may intersect with these rows too, but as they are thinned to one pixel wide, those intersections will not add significantly to our totals. The Numpy `sum()` function makes it easy to perform these additions along the two axes of our array.

Note that the single pixel wide straight lines will wander slightly between a few rows or columns. The thinning math does not guarrantee a perfectly straight line and so a vertical thinned line may cross from one column to the next a few times. That results in lines that look a little jagged along their length, where the line hops from one column to the next. So when we look at a graph of the sums along the vertical or horizontal axis, lines are visible a bumps rather than spikes, as they contribute their "ones" to a few rows or columns.

## Making a Custom Template

At this stage it is possible to pick a representative image and make your own form template, using sums of the ink along the X or Y axes. One way to do this is to create your lists of sums, then draw a colored line on top of the image from the correct position on the X or Y axis. Draw a line with a length that corresponds to the quanitity of pixels with ink in the same row or column. You are essentially creating a bar graph that overlays the image. This will allow you to easily see the location of form lines. We saved such an image, then opend it in an editor in order to read out the exact X or Y pixel locations for each line.

## Positioning the Template of Vertical Lines

We manually created a "template" of the positions of vertical lines along the X axis from a representative form image. These numbers were placed in an array in the Constants section above. Now it is time to use those numbers to find the matching vertical lines in our image. We want to be able to identify particular cells within the form, based on the application of this template. For instance the demographic column is located between the tenth and eleventh vertical line in our template. Of course the template is based on a different image. We will have to adjust the positions of the template lines to match this image. To do that we are only allowed to both stretch or magnify the template and shift the whole template from left to right. These are the ways in which the original pages may have been distorted or photographed differently, i.e. placement on the mat and different camera's zoom settings.

How do we find the proper magnification factor and the right amount left or right to shift the vertical line positions? The answer is that we evaluate a range of possible solutions and see which combination of magnification and shift allows the template to capture the most "ones" or ink at the adjusted vertical line positions. The best solution is the one where the template overlaps the most with the printed vertical lines in our form. Since the skeleton lines are jagged, crossing column boundaries sometimes, we will allow each template line to capture the ink from neighboring columns too.

We will write a function to calculate the "ink captured" by each adjusted template, saving only the best performing magnification and shift values as we go. We will restrict our shifting and magifying range to reasonable values. It is not possible for the template to be shifted more than say 1/5 of the page width, for instance. In the shift and magnification ranges we have to test are quite small.

The `maximize_templated_ink()` function takes in two mandatory and two optional parameters. The first parameter is a one-dimension array of the sum of all of the "ones" or ink along the Y axis. The second parameter is the array that gives the original template line positions. Then we have an option range for translation or shift (given as a tuple) and another optional range for magnification. There are some reasonable defaults, in case a range is not specified. The return value will be the combination of shift and magification that best fit the template to the data, i.e. the ink. The outer for loop explores the range of magnifications, then the inner for loop explores all possible translation values for each given magnification in turn. For each combination a `found_ink` value is calculated and compared with the highest `found_ink` value so far discovered (`max_ink`). If `found_ink` is more than `max_ink`, then it becomes the new `max_ink` going forward. The result is set to the combination of values that produces the new `max_ink`. When these loops finish, we have a magnification and translation that maximizes the ink that is covered by our vertical line template.

In [ ]:
def maximize_templated_ink(ink, template, translation=(-35,35), zoom_px=(-10, 5) ):
    result = (0,0)
    max_ink = 0
    for translation in range(translation[0], translation[1]):
        for zpx in range(zoom_px[0], zoom_px[1]):  # zoom range in pixels add/subtracted for entire dimension
            found_ink = 0
            zoom_factor = zpx/len(ink)  # zoom = zpx / total pixels (in this dimension only)
            for l in template:  # l represents each line position in the template
                l_z = int(l + l*zoom_factor)  # template line is "zoomed"
                test_offset = l_z + translation  # "zoomed" line is translated (shifted left or right)
                try: 
                    # This line's ink is contributed to the total found ink at this zoom and translation
                    # Neighbor ink is also included
                    found_ink = found_ink + ink[test_offset] + ink[test_offset-1] + ink[test_offset+1]
                except IndexError:  # We are ignoring any errors due to line tests beyond the range of our ink array..
                    print(test_offset)
                    pass
            if found_ink > max_ink:  # If we found more ink than before, set a new max and new result.
                max_ink = found_ink
                result = (zoom_factor, translation)
    return result  # return the combination that yeilded maximum ink

We need to adjust our `extract()` function to position the template and give us a good debug view of the result.

In [ ]:
def extract(image, filename, debug=False):
    result = []
    height, width, depth = image.shape
    height = int(height/2)
    width = int(width/2)
    image = cv2.resize(image, (width, height))
    grayimage = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    stencil = get_border_stencil(grayimage, debug=False)

    # Otsu threshold and invert image in one step. Once again threshold based on middle of image.
    middlegrayimg = grayimage[int(height/4):int(height*3/4), int(width/4):int(width*3/4)]
    threshold = cv2.threshold(middlegrayimg, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[0]
    binary_inverted_img = cv2.threshold(grayimage, threshold, 255, cv2.THRESH_BINARY_INV)[1]

    # Apply the border mask with a bitwise AND operation
    binary_inverted_img = cv2.bitwise_and(binary_inverted_img, binary_inverted_img, mask=stencil)
    if debug and False:
        view(binary_inverted_img, "inverted binary image without mat")
    
    # The zero specifies Zhuang Suen algorithm to the thinning operation, as per NCSA paper
    skeleton = cv2.ximgproc.thinning(binary_inverted_img, 0)
    if debug and False:
        view(skeleton[int(height/2)-500:int(height/2)+500, int(width/2)-500:int(width/2)+500], "the skeleton (zoomed in)")

    radians = get_radians_to_unrotate(skeleton, debug=False)
    if debug and False:
        print(f"The rotate angle for {filename} is {radians*180/np.pi} degrees.")

    # We want to fix the rotation in all of our versions of the image..
    skeleton = rotate_image(skeleton, radians)
    binary_inverted_img = rotate_image(binary_inverted_img, radians)
    image = rotate_image(image, radians)

    # Position the vertical template.
    v_lines_threshold = image.shape[1] / 7  # ink threshold is a proportion of image dimension
    v_ink = np.sum(skeleton, axis=0)  # Sum the ink along the Y axis, giving us a total for each position on the X axis.
    v_ink_thresh = [ x if x > v_lines_threshold else 0 for x in v_ink ]  # Zero out any ink below our threshold value.
    vzfactor, v_ink_offset = maximize_templated_ink(v_ink_thresh, vert_lines)  # search for best template position
    my_v_lines = [ int(l+l*vzfactor)+v_ink_offset for l in vert_lines]  # adjust template
    if debug:
        for l in my_v_lines:
            cv2.line(image, (l, 0), (l, image.shape[0]), RED, 1, cv2.LINE_AA)
        view(image, f"vertical template drawn at {vzfactor}, {v_ink_offset} for {filename}")

    return result

Okay, so one additional step not mentioned is that we focused our search for ink on those lines with a lot of ink, ignoring the ink in columns with only a little bit. Our threshold is one seventh of the vertical size of the image. This eliminates some noise from extraneous lines. In the debug view we should the vertical template lines in their new positions in red and give the zoom and translation used.

In [ ]:

run('pages', page_range=(2,17), debug=True)

Great! Hopefully you see the same output as me, which is crisp vertical red lines that mostly line up exactly with the printed vertical lines on the printed page. The red lines represent the pixel offsets that will allow us to pick out vertical slices, i.e. columns within the printed form. Of course the next thing that we need is the rows as well. Then we can extract individual cell images from the page.

## Horizontal Row Template and Alignment Challenges

If you look at the census pages, you may note that the middle, main table of individuals, is at different vertical positions, depending upon the page. In some cases the notes field is about the main table, in other cases it is below it. The notes field also seems to vary in size from page to page. To tackle this problem we are going to create a horizontal lines template that only includes the lines that make up the main table, ignoring the lines in the header and the footer of the page. We hope that our `maximize_templated_ink()` algorithm will still find the most ink when that template is matched to the main table.

You may also have noted earlier the `maximize_templated_ink()` function does not have a parameter for which dimension it is operating over, X or Y. That function works on a single dimension, i.e. a linear distribution of summed ink and a linear list of template offsets. So the same function should work for finding horizontal lines..

In [ ]:
def extract(image, filename, debug=False):
    result = []
    height, width, depth = image.shape
    height = int(height/2)
    width = int(width/2)
    image = cv2.resize(image, (width, height))
    grayimage = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    stencil = get_border_stencil(grayimage, debug=False)

    # Otsu threshold and invert image in one step. Once again threshold based on middle of image.
    middlegrayimg = grayimage[int(height/4):int(height*3/4), int(width/4):int(width*3/4)]
    threshold = cv2.threshold(middlegrayimg, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[0]
    binary_inverted_img = cv2.threshold(grayimage, threshold, 255, cv2.THRESH_BINARY_INV)[1]

    # Apply the border mask with a bitwise AND operation
    binary_inverted_img = cv2.bitwise_and(binary_inverted_img, binary_inverted_img, mask=stencil)
    if debug and False:
        view(binary_inverted_img, "inverted binary image without mat")
    
    # The zero specifies Zhuang Suen algorithm to the thinning operation, as per NCSA paper
    skeleton = cv2.ximgproc.thinning(binary_inverted_img, 0)
    if debug and False:
        view(skeleton[int(height/2)-500:int(height/2)+500, int(width/2)-500:int(width/2)+500], "the skeleton (zoomed in)")

    radians = get_radians_to_unrotate(skeleton, debug=False)
    if debug and False:
        print(f"The rotate angle for {filename} is {radians*180/np.pi} degrees.")

    # We want to fix the rotation in all of our versions of the image..
    skeleton = rotate_image(skeleton, radians)
    binary_inverted_img = rotate_image(binary_inverted_img, radians)
    image = rotate_image(image, radians)

    # Position the vertical template.
    v_lines_threshold = image.shape[1] / 7  # ink threshold is a proportion of image dimension
    v_ink = np.sum(skeleton, axis=0)  # Sum the ink along the Y axis, giving us a total for each position on the X axis.
    v_ink_thresh = [ x if x > v_lines_threshold else 0 for x in v_ink ]  # Zero out any ink below our threshold value.
    vzfactor, v_ink_offset = maximize_templated_ink(v_ink_thresh, vert_lines)  # search for best template position
    my_v_lines = [ int(l+l*vzfactor)+v_ink_offset for l in vert_lines]  # adjust template

    h_lines_threshold = image.shape[0] / 7  # ink threshold is a proportion of image dimension
    h_ink = np.sum(skeleton, axis=1)  # Note that we are using a different axis here and the line above.
    h_ink_thresh = [ x if x > h_lines_threshold else 0 for x in h_ink ]  # Only include ink above the threshold level
    hzfactor, h_ink_offset = maximize_templated_ink(h_ink_thresh, horiz_lines)
    my_h_lines = [ int(l+l*vzfactor)+h_ink_offset for l in horiz_lines ]

    if debug:
        for l in my_h_lines:
            cv2.line(image, (0, l), (image.shape[1], l), RED, 1, cv2.LINE_AA)
        for l in my_v_lines:
            cv2.line(image, (l, 0), (l, image.shape[0]), RED, 1, cv2.LINE_AA)
        view(image, f"horiz template drawn at {hzfactor}, {h_ink_offset} for {filename}")

    return result

The h_ink related code is almost identical to the v_ink related code, except that a different axis is used throughout. Debug code has been added to draw the aligned template of horizontal red lines as well.

In [ ]:

run('pages', page_range=(2,17), debug=True)

If you look at where the horizontal lines are drawn, you will see that they line up with printed lines in many cases, but perhaps a third of them are off the mark. We need to do something to home in on the correct alignment, taking advantage of whatever information we can find in the images. One commonality for the image with problems is that the printing if fairly faint. Lines towards the botton of the page are sometimes nearly invisible. To look into alignment issues further you would want to look in detail at the skeleton images the last step was based upon, instead of the original image. If you are curious you can use that as your debug image instead.

## Help from Detection of a Consistent Image Feature

So then, what other features in these page images are prominent and have a consistent location with respect to the main table? If you look carefully then you may find several visually distinctive clues, such as the boxes that extend on the left and right side. These indicate people in the census who were asked to complete a longer information form. Unfortunately those boxes are in different vertical positions on different pages! That doesn't make them our first choice.

Every form has one vertical line that is significantly darker and thicker than the rest, roughly in the middle of the document and a part of the main table. While the top of that heavy line is well printed on most pages, the bottom is sometimes faded or not printed well. Next we will try to detect the position of the vertical heavy line on every page and use the location of the top of that line as a landmark for the alignment of our horizontal line template.  See the example image below:

<img src="pages/43290879-California-101393-0006.jpeg" height="500"/>

Our next step is to create a function that will detect the shape of that heavy vertical line and return the position of its top along the Y axis.

In [ ]:
# Find significant vertical lines with a matching rectangular kernel shape.
# Input image is our inverted binary image with the mat removed.
def detectDarkVertLine(image, debug=False):
    debug_image = None
    if debug:
        debug_image = cv2.cvtColor(image,cv2.COLOR_GRAY2BGR)

    # First we heal the breaks in any thick vertical lines
    dkernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 7))
    dilated = cv2.dilate(image, dkernel)

    # A 2 x 50 rectangle is used as a kernel to detect heavy vertical lines.
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 50))
    remove_vertical = cv2.morphologyEx(dilated, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
    cnts = cv2.findContours(remove_vertical, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    
    # Of the shapes detected, find the one that is the longest vertically
    rect = None
    for c in cnts:
        r = cv2.boundingRect(c)
        (x, y, w, h) = r
        if debug:
            cv2.rectangle(debug_image, r, BLUE, thickness=2)
        if rect is None or r[3] > rect[3]:
            rect = r

    if debug:
        cv2.rectangle(debug_image, rect, RED, thickness=2)
        view(debug_image, "Dilated image with contour bounding rectangles in blue, vertically longest in red.")
    return rect[1]

Okay, now we need to revise our `extract()` function once again to call this new function.

In [ ]:
def extract(image, filename, debug=False):
    result = []
    height, width, depth = image.shape
    height = int(height/2)
    width = int(width/2)
    image = cv2.resize(image, (width, height))
    grayimage = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    stencil = get_border_stencil(grayimage, debug=False)

    # Otsu threshold and invert image in one step. Once again threshold based on middle of image.
    middlegrayimg = grayimage[int(height/4):int(height*3/4), int(width/4):int(width*3/4)]
    threshold = cv2.threshold(middlegrayimg, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[0]
    binary_inverted_img = cv2.threshold(grayimage, threshold, 255, cv2.THRESH_BINARY_INV)[1]

    # Apply the border mask with a bitwise AND operation
    binary_inverted_img = cv2.bitwise_and(binary_inverted_img, binary_inverted_img, mask=stencil)
    if debug and False:
        view(binary_inverted_img, "inverted binary image without mat")
    
    # The zero specifies Zhuang Suen algorithm to the thinning operation, as per NCSA paper
    skeleton = cv2.ximgproc.thinning(binary_inverted_img, 0)
    if debug and False:
        view(skeleton[int(height/2)-500:int(height/2)+500, int(width/2)-500:int(width/2)+500], "the skeleton (zoomed in)")

    radians = get_radians_to_unrotate(skeleton, debug=False)
    if debug and False:
        print(f"The rotate angle for {filename} is {radians*180/np.pi} degrees.")

    # We want to fix the rotation in all of our versions of the image..
    skeleton = rotate_image(skeleton, radians)
    binary_inverted_img = rotate_image(binary_inverted_img, radians)
    image = rotate_image(image, radians)

    # Position the vertical template.
    v_lines_threshold = image.shape[1] / 7  # ink threshold is a proportion of image dimension
    v_ink = np.sum(skeleton, axis=0)  # Sum the ink along the Y axis, giving us a total for each position on the X axis.
    v_ink_thresh = [ x if x > v_lines_threshold else 0 for x in v_ink ]  # Zero out any ink below our threshold value.
    vzfactor, v_ink_offset = maximize_templated_ink(v_ink_thresh, vert_lines)  # search for best template position
    my_v_lines = [ int(l+l*vzfactor)+v_ink_offset for l in vert_lines]  # adjust template

    dark_vert_line_top = detectDarkVertLine(binary_inverted_img, debug=debug)  # Find the top of the heavy vertical line..
    # The top of the heavy line corresponds to the first line in the horizontal template
    # So the translation values to explore should position the first line near there.
    # 't' below is the ideal translation for the heavy line, around which we will try to find max ink again.
    t = dark_vert_line_top - horiz_lines[0]  

    h_lines_threshold = image.shape[0] / 7  # ink threshold is a proportion of image dimension
    h_ink = np.sum(skeleton, axis=1)  # Note that we are using a different axis here and the line above.
    h_ink_thresh = [ x if x > h_lines_threshold else 0 for x in h_ink ]  # Only include ink above the threshold level
    # Below we explore a very limited range of translation that puts the first line at the top of the heavy vertical.
    hzfactor, h_ink_offset = maximize_templated_ink(h_ink_thresh, horiz_lines, translation=(t-2, t+2))
    my_h_lines = [ int(l+l*hzfactor)+h_ink_offset for l in horiz_lines ]

    if debug:
        for l in my_h_lines:
            cv2.line(image, (0, l), (image.shape[1], l), RED, 1, cv2.LINE_AA)
        for l in my_v_lines:
            cv2.line(image, (l, 0), (l, image.shape[0]), RED, 1, cv2.LINE_AA)
        view(image, f"horiz template drawn at {hzfactor}, {h_ink_offset} for {filename}")

    return result

In [ ]:

run('pages', page_range=(2,17), debug=True)

Success! If you inspect the results, by and large these templates are well-fitted to the printed forms..

A close inspection of our results shows that almost all of the table templates are properly positioned. I think that we have one remaining issue in certain images. In a select few images the zoom factor, or magnification, seems to be a little off.. We see that most of the rows are drawn correctly, but as you go down the page the lines stray more and more from the printed page. You can see this problem in a pronounced way on example image 7, show below. In image 7 the template is almost a full row off the mark by the bottom of the table.

In [ ]:

run('pages', page_range=(7,8), debug=True)

Since this problem is likely a zoom issue, what if we try reusing the zoom factor that was discovered in the alignment of the horizontal lines? Shouldn't the vertical and horizontal zoom be the same? Let's try that next. We will use our zoom_px optional tuple parameter to tell the maximize function to enlarge the template no more than the magnification that was found for the vertical lines template.

In [ ]:
def extract(image, filename, debug=False):
    result = []
    height, width, depth = image.shape
    height = int(height/2)
    width = int(width/2)
    image = cv2.resize(image, (width, height))
    grayimage = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    stencil = get_border_stencil(grayimage, debug=False)

    # Otsu threshold and invert image in one step. Once again threshold based on middle of image.
    middlegrayimg = grayimage[int(height/4):int(height*3/4), int(width/4):int(width*3/4)]
    threshold = cv2.threshold(middlegrayimg, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[0]
    binary_inverted_img = cv2.threshold(grayimage, threshold, 255, cv2.THRESH_BINARY_INV)[1]

    # Apply the border mask with a bitwise AND operation
    binary_inverted_img = cv2.bitwise_and(binary_inverted_img, binary_inverted_img, mask=stencil)
    if debug and False:
        view(binary_inverted_img, "inverted binary image without mat")
    
    # The zero specifies Zhuang Suen algorithm to the thinning operation, as per NCSA paper
    skeleton = cv2.ximgproc.thinning(binary_inverted_img, 0)
    if debug and False:
        view(skeleton[int(height/2)-500:int(height/2)+500, int(width/2)-500:int(width/2)+500], "the skeleton (zoomed in)")

    radians = get_radians_to_unrotate(skeleton, debug=False)
    if debug and False:
        print(f"The rotate angle for {filename} is {radians*180/np.pi} degrees.")

    # We want to fix the rotation in all of our versions of the image..
    skeleton = rotate_image(skeleton, radians)
    binary_inverted_img = rotate_image(binary_inverted_img, radians)
    image = rotate_image(image, radians)

    # Position the vertical template.
    v_lines_threshold = image.shape[1] / 7  # ink threshold is a proportion of image dimension
    v_ink = np.sum(skeleton, axis=0)  # Sum the ink along the Y axis, giving us a total for each position on the X axis.
    v_ink_thresh = [ x if x > v_lines_threshold else 0 for x in v_ink ]  # Zero out any ink below our threshold value.
    vzfactor, v_ink_offset = maximize_templated_ink(v_ink_thresh, vert_lines)  # search for best template position
    my_v_lines = [ int(l+l*vzfactor)+v_ink_offset for l in vert_lines]  # adjust template

    dark_vert_line_top = detectDarkVertLine(binary_inverted_img, debug=False)  # Find the top of the heavy vertical line..
    # The top of the heavy line corresponds to the first line in the horizontal template
    # So the translation values to explore should position the first line near there.
    # 't' below is the ideal translation for the heavy line, around which we will try to find max ink again.
    t = dark_vert_line_top - horiz_lines[0]  

    # Reusing vzfactor in our search for horizontal lines
    zpx = int(vzfactor * image.shape[0])

    h_lines_threshold = image.shape[0] / 7  # ink threshold is a proportion of image dimension
    h_ink = np.sum(skeleton, axis=1)  # Note that we are using a different axis here and the line above.
    h_ink_thresh = [ x if x > h_lines_threshold else 0 for x in h_ink ]  # Only include ink above the threshold level
    # Below we explore a very limited range of translation that puts the first line at the top of the heavy vertical.
    # We are also now constraining the zoom, with a max zoom equal to the zoom of the vertical lines template.
    hzfactor, h_ink_offset = maximize_templated_ink(h_ink_thresh, horiz_lines, translation=(t-2, t+2), zoom_px=(zpx-10, zpx))
    my_h_lines = [ int(l+l*hzfactor)+h_ink_offset for l in horiz_lines ]

    if debug:
        for l in my_h_lines:
            cv2.line(image, (0, l), (image.shape[1], l), RED, 1, cv2.LINE_AA)
        for l in my_v_lines:
            cv2.line(image, (l, 0), (l, image.shape[0]), RED, 1, cv2.LINE_AA)
        view(image, f"horiz template drawn at {hzfactor}, {h_ink_offset} for {filename}")

    return result

In [ ]:

run('pages', page_range=(2,16), debug=True)

Huzzah! Now our red template lines are well fitted to every row and column in the main table for every image in our example pages. This means we have all the information that we need to extract smaller, cell-specific, images for further analysis. As a convenience we will now modify the extract method such that it returned a neatly packaged result.

In [ ]:
def extract(image, filename, debug=False):
    result = []
    height, width, depth = image.shape
    height = int(height/2)
    width = int(width/2)
    image = cv2.resize(image, (width, height))
    grayimage = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    stencil = get_border_stencil(grayimage, debug=False)

    # Otsu threshold and invert image in one step. Once again threshold based on middle of image.
    middlegrayimg = grayimage[int(height/4):int(height*3/4), int(width/4):int(width*3/4)]
    threshold = cv2.threshold(middlegrayimg, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[0]
    binary_inverted_img = cv2.threshold(grayimage, threshold, 255, cv2.THRESH_BINARY_INV)[1]

    # Apply the border mask with a bitwise AND operation
    binary_inverted_img = cv2.bitwise_and(binary_inverted_img, binary_inverted_img, mask=stencil)
    if debug and False:
        view(binary_inverted_img, "inverted binary image without mat")
    
    # The zero specifies Zhuang Suen algorithm to the thinning operation, as per NCSA paper
    skeleton = cv2.ximgproc.thinning(binary_inverted_img, 0)
    if debug and False:
        view(skeleton[int(height/2)-500:int(height/2)+500, int(width/2)-500:int(width/2)+500], "the skeleton (zoomed in)")

    radians = get_radians_to_unrotate(skeleton, debug=False)
    if debug and False:
        print(f"The rotate angle for {filename} is {radians*180/np.pi} degrees.")

    # We want to fix the rotation in all of our versions of the image..
    skeleton = rotate_image(skeleton, radians)
    binary_inverted_img = rotate_image(binary_inverted_img, radians)
    image = rotate_image(image, radians)

    # Position the vertical template.
    v_lines_threshold = image.shape[1] / 7  # ink threshold is a proportion of image dimension
    v_ink = np.sum(skeleton, axis=0)  # Sum the ink along the Y axis, giving us a total for each position on the X axis.
    v_ink_thresh = [ x if x > v_lines_threshold else 0 for x in v_ink ]  # Zero out any ink below our threshold value.
    vzfactor, v_ink_offset = maximize_templated_ink(v_ink_thresh, vert_lines)  # search for best template position
    my_v_lines = [ int(l+l*vzfactor)+v_ink_offset for l in vert_lines]  # adjust template

    dark_vert_line_top = detectDarkVertLine(binary_inverted_img, debug=False)  # Find the top of the heavy vertical line..
    # The top of the heavy line corresponds to the first line in the horizontal template
    # So the translation values to explore should position the first line near there.
    # 't' below is the ideal translation for the heavy line, around which we will try to find max ink again.
    t = dark_vert_line_top - horiz_lines[0]  

    # Reusing vzfactor in our search for horizontal lines
    zpx = int(vzfactor * image.shape[0])

    h_lines_threshold = image.shape[0] / 7  # ink threshold is a proportion of image dimension
    h_ink = np.sum(skeleton, axis=1)  # Note that we are using a different axis here and the line above.
    h_ink_thresh = [ x if x > h_lines_threshold else 0 for x in h_ink ]  # Only include ink above the threshold level
    # Below we explore a very limited range of translation that puts the first line at the top of the heavy vertical.
    # We are also now constraining the zoom, with a max zoom equal to the zoom of the vertical lines template.
    hzfactor, h_ink_offset = maximize_templated_ink(h_ink_thresh, horiz_lines, translation=(t-2, t+2), zoom_px=(zpx-10, zpx))
    my_h_lines = [ int(l+l*hzfactor)+h_ink_offset for l in horiz_lines ]

    if debug and False:
        for l in my_h_lines:
            cv2.line(image, (0, l), (image.shape[1], l), RED, 1, cv2.LINE_AA)
        for l in my_v_lines:
            cv2.line(image, (l, 0), (l, image.shape[0]), RED, 1, cv2.LINE_AA)
        view(image, f"horiz template drawn at {hzfactor}, {h_ink_offset} for {filename}")

    return (image, my_v_lines, my_h_lines)  # returning the rotated image and the adjusted template

Now let us modify our run function to use this information for each image. Let's take the example of the demographic column from the census form. The run function below will outline each demographic cell in red.

In [ ]:
def run(path, page_range=(2, 16), debug=False):
    for f in sorted(glob.glob(f'{path}/*')):
        pagestr = re.search(r'-(\d+).jpeg', f).group(1)
        if int(pagestr) not in range(page_range[0], page_range[1]):
            continue
        image = cv2.imread(f)
        (adjusted_img, v_lines, h_lines) = extract(image, f, debug=debug)
        demo_h_offset = v_lines[11]  # the demographic column starts the the 12th vertical line
        demo_width = v_lines[12] - demo_h_offset  # width calculation
        # Open CV rectangles calculation for each demographic cell
        demographic_cells = \
          [ (demo_h_offset, h_lines[i], demo_width, int(h_lines[i+1]-h_lines[i])) for i in range(3, len(h_lines)-1)]
        for r in demographic_cells:
            cv2.rectangle(adjusted_img, r, RED, thickness=2)
        view(adjusted_img, "cells outlined in red")

run('pages', page_range=(2,16))


## Review

This form segmentation workflow was composed from many separate steps that used a number of different image processing algorithms:

* Thresholding - Making a binary image from a grayscale one.
* Shape Detection (Morphological Transform) - Using a kernel shape to detect other shapes.
* Shape Thinning (Skeletonize) - Thinning shapes to a thickness of a single pixel.
* Hough Lines Transform - Detection of straight lines anywhere in an image.
* Custom Form Deskewing - Rotating a printed form to align it with the image X and Y axes.
* Custom Feature Detection - Detecting a distinctive image feature and using its location.
* Fitting a Line Template - Finding the best position and zoom factor for a form-specific template.
* Debugging Views - Creating debug images that show the work in progress by drawing on images.

This notebook used the Python library for OpenCV 2, which is freely available for download and reuse in your own image processing workflows. Many resources were consulted in the construction of this notebook, including the NCSA paper on image segmentation in the 1940s census forms and Stack Exchange, which has many helpful answered questions on OpenCV techniques.